RAG Configuration in this notebook:

Embedding model: thenlper/gte-base

parent_splitter = chunk_size=1000,chunk_overlap=100

child_splitter = chunk_size=200,chunk_overlap=20


Generation Model: llama-3-8B-Instruct

Retriever: ParentChildRetriever

Embedding Size: 768

In [ ]:
import pickle
from pinecone import Pinecone, ServerlessSpec
import getpass
import os
import time
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import pinecone
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import Index
from langchain_pinecone import PineconeVectorStore
from transformers import BitsAndBytesConfig
import bitsandbytes
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
from tqdm import tqdm
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
import json
from langchain.storage._lc_store import create_kv_docstore
import re


In [2]:
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
#pcsk_zaxjG_6efuvnfQGiuFKbKtKPfgtFvdHu1CEkoCwQEEsBDS8HC84mh5VRvmQREvnr4oF7N

Enter your Pinecone API key: ··········


In [ ]:
# Embedding model
model_name = "thenlper/gte-base"
model_kwargs = {"device": "cuda" if torch.cuda.is_available() else "cpu"}
encode_kwargs = {"normalize_embeddings": False}
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

<ipython-input-6-1b00d0fc66cd>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
!pip install -U bitsandbytes

In [ ]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

llm_model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", quantization_config=bnb_config)
llm_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [3]:
# Create the index if it doesn't exist
#pc.create_index(name="rag-llm-pc-retriever",
#                   dimension=768,metric="cosine",
#                   spec=ServerlessSpec(
#                    cloud="aws",
#                    region="us-east-1"))

pc_index = pc.Index("rag-llm-pc-retriever")

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Load the documents
with open("data_5983_updated.pkl", "rb") as file:
    documents = pickle.load(file)

In [ ]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)

child_splitter = RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)

from langchain.storage import InMemoryStore
store = InMemoryStore()

In [ ]:
# Storage layer for the parent chunks
store = InMemoryStore()

In [ ]:
store

In [ ]:
from langchain.schema import Document

parent_documents = []
for doc in documents:
    parent_chunks = parent_splitter.split_text(doc.page_content)
    for i, chunk in enumerate(parent_chunks):
        parent_documents.append(Document(
            id=f"{doc.metadata['id']}_parent_{i}",
            page_content=chunk,
            metadata=doc.metadata
        ))

In [ ]:
parent_doc_path = "C:/Users/91998/Desktop/LLM_Project/id_parent_documents"

if not os.path.exists(parent_doc_path):
    os.makedirs(parent_doc_path)

for doc in parent_documents:
    doc_id = doc.id
    file_path = os.path.join(parent_doc_path, f"{doc_id}.json")

    document_content = {
        "id": doc.id,
        "text": doc.page_content,
        "metadata": doc.metadata
    }

    with open(file_path, "w") as f:
        json.dump(document_content, f)

In [ ]:
from langchain.storage import LocalFileStore
from langchain.storage._lc_store import create_kv_docstore

fs = LocalFileStore(parent_doc_path)

store = create_kv_docstore(fs)

In [ ]:
store

In [ ]:
child_documents = []
for parent_doc in parent_documents:
    child_chunks = child_splitter.split_text(parent_doc.page_content)
    for i, chunk in enumerate(child_chunks):
        child_documents.append(Document(
            id=f"{parent_doc.id}_child_{i}",
            page_content=chunk,
            metadata=parent_doc.metadata
        ))

In [ ]:
# VectorStore
vectorstore = PineconeVectorStore(
    index_name="rag-llm-pc-retriever",
    embedding=hf_embeddings,
)

In [4]:
index_stats = pc_index.describe_index_stats()

print("Index Stats:", index_stats)

Index Stats: {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10486}},
 'total_vector_count': 10486}


In [ ]:
store = create_kv_docstore(parent_documents)

In [ ]:
store

In [ ]:
from langchain.retrievers import ParentDocumentRetriever

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)


In [ ]:
retriever.add_documents(documents)

In [ ]:
retriever.invoke("What is PTSD?")

[Document(metadata={'title': 'Post Traumatic Stress Disorder (PTSD)', 'source': 'https://www.samhsa.gov/mental-health/post-traumatic-stress-disorder', 'category': 'Mental Illness', 'id': 'e0709607-37f7-4085-a9c4-220a95994af5'}, page_content='Post-traumatic stress disorder (PTSD) is a real disorder that develops when a person has experienced or witnessed a scary, shocking, terrifying, or dangerous event. These stressful or traumatic events usually involve a situation where someone’s life has been threatened or severe injury has occurred. Children and adults with PTSD may feel anxious or stressed even when they are not in present danger.\nCauses\nYou can get PTSD after living through or seeing a traumatic event, such as war, a natural disaster, sexual assault, physical abuse, or a bad accident. PTSD makes you feel stressed and afraid after the danger is over. It affects your life and the people around you.'),
 Document(metadata={'category': 'Mental Illness', 'source': 'Counsel chat datas

In [ ]:
file_path = 'parent_documents.pkl'

with open(file_path, 'wb') as pickle_file:
    pickle.dump(parent_documents, pickle_file)

In [ ]:
file_path = 'child_documents.pkl'

with open(file_path, 'wb') as pickle_file:
    pickle.dump(child_documents, pickle_file)

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

In [ ]:
llm_tokenizer.pad_token_id = llm_tokenizer.eos_token_id

In [ ]:
llm_pipeline = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=llm_tokenizer,
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=100,
    top_p=0.9,
    top_k=50,
    eos_token_id=llm_tokenizer.eos_token_id
)

In [ ]:
llm_final_model = HuggingFacePipeline(pipeline=llm_pipeline)

<ipython-input-105-3f741d8043ea>:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm_final_model = HuggingFacePipeline(pipeline=llm_pipeline)


In [ ]:
# Prompt template
template = """
You are a compassionate and knowledgeable mental health assistant that answers questions related to mental health.\n
Use the following pieces of retrieved context to provide a helpful and empathetic response to the user's question.\n
Use only the context provided and not any prior knowledge.\n
If you are unsure of the answer, tell that you do not know the answer.\n
Stick to the question and just answer the question in a short manner.\n
Avoid any additional greetings or elaborations.\n

Context: \n
------------------------------------------------------------------------------\n
{context}
------------------------------------------------------------------------------\n
Given the context and without any prior knowledge, answer the below question.\n
Question: {question}
Answer:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm_final_model,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
query = "What is the Four-Fold Breath technique?"

result= qa_chain.invoke(query)

In [ ]:
answer = result['result']
print("Answer:", answer)

Answer: The Four-Fold Breath technique is a Pranayama yoga breathing technique used by [Name] to relax and allow for better sleep. It involves breathing in slowly to the count of 4 and breathing out even slower to the count of 6. Practicing this technique for about 5 minutes, three times a day, helps to focus on nothing except one's breathing and makes it easier to apply during situations when one is angry or upset.


RAG Response generation

In [ ]:
df = pd.read_csv("U_eval_dataset_100q_with_context.csv")

In [ ]:
results = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating Responses"):
    question = row['Question']
    ground_truth = row['Ground_Truth']
    source = row['Source']
    category = row['Category']
    context = row['contexts']

    rag_result = qa_chain.invoke(question)
    rag_response = rag_result['result']

    results.append({
        "Question": question,
        "Ground_Truth": ground_truth,
        "Source": source,
        "Category": category,
        "Context": context,
        "rag_response": rag_response
    })

results_df = pd.DataFrame(results)

In [ ]:
results_df.head()

,Question,Ground_Truth,Source,Category,Context,rag_response
0,What triggers Seasonal Affective Disorder (SAD)?,"SAD is triggered by seasonal changes, especial...",https://www.samhsa.gov/mental-health/seasonal-...,Mental Illness,"['Seasonal affective disorder, or SAD, is a co...",The trigger for Seasonal Affective Disorder (S...
1,What are the health risks of bulimia nervosa?,"Risks include dental decay, dehydration, and s...",https://www.samhsa.gov/mental-health/eating-di...,Mental Illness,['It is common that binge eating will lead to ...,The health risks of bulimia nervosa include:\n...
2,How can someone with mental health problems ma...,By finding trustworthy individuals who provide...,https://www.samhsa.gov/mental-health/how-to-ta...,Mental Health,"[""Anyone can experience mental health problems...",Maintaining a support system is crucial for so...
3,What are the risk factors for schizophrenia?,Risk factors include genetic predisposition an...,https://www.samhsa.gov/mental-health/schizophr...,Mental Illness,['Schizophrenia is a serious brain disorder th...,"According to the provided context, the risk fa..."
4,How does schizophrenia impact daily functioning?,"It affects work, socialization, and completing...",https://www.samhsa.gov/mental-health/schizophr...,Mental Illness,['Schizophrenia is a serious brain disorder th...,Schizophrenia can be extremely disruptive to a...


In [ ]:
results_df.to_csv("pc-retriever-gte-base-responses.csv")

Evaluation

In [ ]:
import pandas as pd
eval_df = pd.read_csv("pc-retriever-gte-base-responses.csv")

In [ ]:
eval_df.head()

,Question,Ground_Truth,Source,Category,Context,rag_response
0,What triggers Seasonal Affective Disorder (SAD)?,"SAD is triggered by seasonal changes, especial...",https://www.samhsa.gov/mental-health/seasonal-...,Mental Illness,"['Seasonal affective disorder, or SAD, is a co...",The trigger for Seasonal Affective Disorder (S...
1,What are the health risks of bulimia nervosa?,"Risks include dental decay, dehydration, and s...",https://www.samhsa.gov/mental-health/eating-di...,Mental Illness,['It is common that binge eating will lead to ...,The health risks of bulimia nervosa include:\n...
2,How can someone with mental health problems ma...,By finding trustworthy individuals who provide...,https://www.samhsa.gov/mental-health/how-to-ta...,Mental Health,"[""Anyone can experience mental health problems...",Maintaining a support system is crucial for so...
3,What are the risk factors for schizophrenia?,Risk factors include genetic predisposition an...,https://www.samhsa.gov/mental-health/schizophr...,Mental Illness,['Schizophrenia is a serious brain disorder th...,"According to the provided context, the risk fa..."
4,How does schizophrenia impact daily functioning?,"It affects work, socialization, and completing...",https://www.samhsa.gov/mental-health/schizophr...,Mental Illness,['Schizophrenia is a serious brain disorder th...,Schizophrenia can be extremely disruptive to a...


In [ ]:
eval_df = eval_df.to_dict(orient='records')

In [ ]:
# Evaluation prompt
evaluation_prompt = """
### Task Description:
You are provided with an instruction (or query), a response to evaluate, a reference answer that represents an ideal response, and evaluation criteria. Additionally, you are given responses from a RAG model: a RAG response. Your task is to evaluate and compare the RAG response with the reference answer based on the provided criteria.

1. Assess the quality of the response strictly based on the given rubric and criteria (relevance, semantic similarity, accuracy, factual correctness, completeness, clarity, and conciseness), not general impressions.
2. Write short feedback for each response based on the rubric, highlighting strengths and weaknesses for each criterion.
3. Provide feedback in the specified format.
4. Finally, assign an overall score out of 10 for RAG response.


### The instruction to evaluate:
{Question}

### RAG Response:
{rag_response}

### Reference Answer:
{Ground_Truth}

### Rubric for Individual Criteria:
- **Relevance**: How effectively does the response address the specific requirements or intent of the query?
- **Semantic Similarity**: To what extent does the response capture the meaning or intent of the reference answer, even if the phrasing differs?
- **Accuracy**: Does the response provide correct and logically sound information in relation to the query?
- **Factual Correctness**: Are the facts presented in the response verified and consistent with reliable sources or the reference answer?
- **Completeness**: Does the response thoroughly cover all essential aspects of the query without omitting critical information?
- **Clarity**: Is the response expressed in a manner that is straightforward, unambiguous, and easy to comprehend?
- **Conciseness**: Does the response convey the necessary information succinctly, avoiding extraneous details or redundancy?

### Output Format:
Feedback:
- RAG Response:
    - Relevance: {{feedback}}
    - Semantic Similarity: {{feedback}}
    - Accuracy: {{feedback}}
    - Factual Correctness: {{feedback}}
    - Completeness: {{feedback}}
    - Clarity: {{feedback}}
    - Conciseness: {{feedback}}

### Overall Scores:
- RAG Overall Score: {{integer between 1 and 10}}
"""

In [ ]:
import openai

In [ ]:
!pip install openai

In [ ]:
!pip install httpx==0.27.2

In [ ]:
from openai import OpenAI

client = OpenAI(api_key = "YOUR_API_KEY") #Insert your api key here

In [ ]:
import re

def evaluate_responses(instruction, rag_response, reference_answer):
    prompt = evaluation_prompt.format(
        Question=instruction,
        rag_response=rag_response,
        Ground_Truth=reference_answer
    )

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant evaluating model responses."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000,
        temperature=0.5
    )

    s = response.choices[0].message.content
    print("Evaluation Output:")
    print(s)

    rag_score = re.search(r'RAG Overall Score: (\d+)', s).group(1).strip()

    return int(rag_score)

evaluations = []
for data in tqdm(eval_df, desc="Evaluating responses", unit="sample"):
    instruction = data["Question"]
    rag_response = data["rag_response"]
    reference_answer = data["Ground_Truth"]

    rag_score = evaluate_responses(
        instruction, rag_response, reference_answer
    )

    evaluations.append({
        "Question": instruction,
        "Ground_Truth": reference_answer,
        "Source": data["Source"],
        "Category": data["Category"],
        "Context": data["Context"],
        "rag_response": rag_response,
        "RAG_Score": rag_score,
    })

Evaluating responses:   1%|          | 1/100 [00:03<05:41,  3.45s/sample]

Evaluation Output:
Feedback:
- RAG Response:
    - Relevance: The response is relevant as it directly addresses the triggers of Seasonal Affective Disorder (SAD) by mentioning seasonal changes, which is the core aspect of the query.
    - Semantic Similarity: The response captures the essential meaning of the reference answer, though it includes additional context about the timing of SAD, which slightly alters the focus but remains aligned with the original intent.
    - Accuracy: The information provided is accurate, correctly linking SAD to changes in seasons and the impact of sunlight.
    - Factual Correctness: The facts are correct and consistent with the reference answer, confirming that SAD is most prevalent during fall and winter due to reduced sunlight.
    - Completeness: The response is complete; it explains not only the trigger but also the seasonal pattern of SAD, making it informative.
    - Clarity: The response is clear and easy to understand, with straightforward langu

Evaluating responses:   2%|▏         | 2/100 [00:06<04:54,  3.01s/sample]

Evaluation Output:
Feedback:
- RAG Response:
    - Relevance: The response is highly relevant as it directly addresses the health risks associated with bulimia nervosa, which is the focus of the query.
    - Semantic Similarity: The response captures the essence of the reference answer but elaborates significantly more. It does not closely match the brevity of the reference but conveys similar meanings.
    - Accuracy: The information provided is accurate regarding the health risks of bulimia nervosa, aligning well with known medical facts.
    - Factual Correctness: The facts presented are consistent with reliable medical sources regarding the risks of bulimia nervosa, confirming their correctness.
    - Completeness: The response is very complete, listing multiple health risks associated with bulimia nervosa, which provides a broader understanding than the reference answer.
    - Clarity: The response is clear and easy to understand, with each risk presented in a straightforward mann

Evaluating responses:   3%|▎         | 3/100 [00:09<04:49,  2.99s/sample]

...

Evaluation Output:
Feedback:
- RAG Response:
    - Relevance: The response is relevant to the query, addressing the causes of social anxiety disorder directly.
    - Semantic Similarity: The RAG response captures the essence of the reference answer but introduces additional factors, making it slightly less aligned in terms of phrasing.
    - Accuracy: The information provided is generally accurate, mentioning genetics and learned behavior as potential causes of social anxiety disorder.
    - Factual Correctness: The facts presented are consistent with known sources regarding social anxiety disorder, aligning with the reference answer.
    - Completeness: The response includes important factors but could be seen as incomplete because it does not mention "negative social experiences" explicitly, which is a key aspect of the reference answer.
    - Clarity: The response is clear and understandable, effectively communicating the main ideas.
    - Conciseness: The response is concise, deliv

Evaluating responses:  99%|█████████▉| 99/100 [04:07<00:02,  2.87s/sample]

Evaluation Output:
Feedback:
- RAG Response:
    - Relevance: The response is relevant as it addresses the signs that indicate a child may need mental health support, which aligns with the query.
    - Semantic Similarity: The response does not capture the full range of signs mentioned in the reference answer, which includes drastic mood swings and severe behavioral changes. Therefore, it lacks some semantic similarity.
    - Accuracy: The signs mentioned are accurate and reflect serious mental health concerns, but they do not encompass the broader range of signs that could indicate a need for support.
    - Factual Correctness: The information provided is factually correct in terms of mental health indicators, but it is limited in scope.
    - Completeness: The response is incomplete as it only lists two signs, whereas the reference answer suggests a broader range of symptoms that may indicate the need for support.
    - Clarity: The response is clear and easy to understand, with stra

Evaluating responses: 100%|██████████| 100/100 [04:09<00:00,  2.50s/sample]

Evaluation Output:
Feedback:
- RAG Response:
    - Relevance: The response is relevant as it addresses biological factors contributing to mental health conditions. However, it could elaborate further on the topic.
    - Semantic Similarity: The response captures the essence of the reference answer, but the phrasing is slightly less formal and complete.
    - Accuracy: The information provided is accurate; genes and brain chemistry are indeed recognized biological factors in mental health.
    - Factual Correctness: The facts presented are correct and consistent with the reference answer.
    - Completeness: The response lacks completeness as it only mentions two factors without exploring any additional biological factors that may also contribute.
    - Clarity: The response is clear and easy to understand, but it could benefit from slightly more detail.
    - Conciseness: The response is concise, effectively conveying the main idea without unnecessary information.

### Overall Scores:


In [ ]:
import json

with open("evaluation_results_rag_pc_retriever_gte-base.json", "w") as f:
    json.dump(evaluations, f, indent=4)

In [ ]:
results = pd.DataFrame(evaluations)

In [ ]:
results[['RAG_Score']].describe()

,RAG_Score
count,100.000000
mean,6.860000
std,1.943755
min,1.000000
25%,6.000000
50%,7.500000
75%,8.000000
max,9.000000


In [ ]:
average_rag_score = results['RAG_Score'].mean()

print("Average RAG Score:", average_rag_score)

Average RAG Score: 6.86
